<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

## 导入相关包，创建相机实例

In [1]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import PID
camera = Camera.instance(width=720, height=720)

## 创建相关控制变量

In [2]:
global face_x, face_y, face_w, face_h
face_x = face_y = face_w = face_h = 0
global target_valuex
target_valuex = 2100
global target_valuey
target_valuey = 2048

## 创建PID控制实例

In [3]:
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

## 创建云台总线舵机实例

In [4]:
from servoserial import ServoSerial
servo_device = ServoSerial() 

SerialException: [Errno 13] could not open port /dev/ttyTHS1: [Errno 13] Permission denied: '/dev/ttyTHS1'

## 创建显示控件

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
face_image = widgets.Image(format='jpeg', width=300, height=300)
display(face_image)

## 载入"Haar"" 级联分类器

In [ ]:
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_profileface.xml')

## 云台运动主进程

In [ ]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale( gray )
    if len(faces)>0:
        (face_x, face_y, face_w, face_h) = faces[0]
        # 将检测到的人脸标记出来
        # cv2.rectangle(frame,(face_x,face_y),(face_x+face_h,face_y+face_w),(0,255,0),2)
        cv2.rectangle(frame,(face_x+10,face_y),(face_x+face_w-10,face_y+face_h+20),(0,255,0),2)

        #Proportion-Integration-Differentiation算法
        # 输入X轴方向参数PID控制输入
        xservo_pid.SystemOutput = face_x+face_h/2
        xservo_pid.SetStepSignal(150)
        xservo_pid.SetInertiaTime(0.01, 0.006)
        target_valuex = int(2100 + xservo_pid.SystemOutput)
        # 输入Y轴方向参数PID控制输入
        yservo_pid.SystemOutput = face_y+face_w/2
        yservo_pid.SetStepSignal(150)
        yservo_pid.SetInertiaTime(0.01, 0.006)
        target_valuey = int(2048+yservo_pid.SystemOutput)
        # 将云台转动至PID调校位置
        servo_device.Servo_serial_double_control(1, target_valuex, 2, target_valuey)
    # 实时传回图像数据进行显示
    face_image.value = bgr8_to_jpeg(frame)